In [1]:
import pandas as pd
import pickle

In [16]:
new_point = { 
    'Yas': 53,
    'Cinsiyet': 'Kadın',
    'Medeni_Hal': 'Bekar',
    'Is_Durumu': 'Çalışıyor',
    'Egitim': 'Lise',
    'Gelir_Duzeyi': 'Düşük',
    'Yasam_Yeri': 'Küçük Şehir',
    'Psikolojik_Destek': 'Hayır',
    'Calisma_Suresi': 12,
    'Soru_1': 1,
    'Soru_2': 4,
    'Soru_3': 2,
    'Soru_4': 4,
    'Soru_5': 3,
    'Soru_6': 3,
    'Soru_7': 4,
    'Soru_8': 3,
    'Soru_9': 4,
    'Soru_10': 4,
    'Soru_11': 4,
    'Soru_12': 2,
    'Soru_13': 1,
    'Soru_14': 0,
    'Soru_15': 3,
    'Soru_16': 1,
    'Soru_17': 4,
    'Soru_18': 3,
    'Soru_19': 1,
    'Soru_20': 1
}

In [17]:
new_data = pd.DataFrame([new_point])

In [18]:
new_data

,Yas,Cinsiyet,Medeni_Hal,Is_Durumu,Egitim,Gelir_Duzeyi,Yasam_Yeri,Psikolojik_Destek,Calisma_Suresi,Soru_1,...,Soru_11,Soru_12,Soru_13,Soru_14,Soru_15,Soru_16,Soru_17,Soru_18,Soru_19,Soru_20
0,53,Kadın,Bekar,Çalışıyor,Lise,Düşük,Küçük Şehir,Hayır,12,1,...,4,2,1,0,3,1,4,3,1,1


In [19]:
# Load the saved encoder
with open('./encoders/anksiyete_rf_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# Load the saved model
with open('./models/anksiyete_rf_model.pkl', 'rb') as f:
    model = pickle.load(f)


In [20]:
# Select only the same categorical columns used during training
categorical_cols = new_data.select_dtypes(include=['object', 'category']).columns

# Encode the categorical part
encoded_new_cat = encoder.transform(new_data[categorical_cols])

# Convert to DataFrame
encoded_new_cat_df = pd.DataFrame(encoded_new_cat, columns=encoder.get_feature_names_out(categorical_cols), index=new_data.index)

# Drop original categorical columns and concatenate encoded ones
new_data_encoded = pd.concat([encoded_new_cat_df,new_data.drop(columns=categorical_cols)], axis=1)


In [21]:
new_data_encoded

,Cinsiyet_Diğer,Cinsiyet_Erkek,Cinsiyet_Kadın,Medeni_Hal_Bekar,Medeni_Hal_Boşanmış,Medeni_Hal_Dul,Medeni_Hal_Evli,Is_Durumu_Emekli,Is_Durumu_Çalışıyor,Is_Durumu_Öğrenci,...,Soru_11,Soru_12,Soru_13,Soru_14,Soru_15,Soru_16,Soru_17,Soru_18,Soru_19,Soru_20
0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,4,2,1,0,3,1,4,3,1,1


In [22]:
# Predict with the loaded model
prediction = model.predict(new_data_encoded)

# Show result
print("Predicted Risk Group:", prediction[0])


Predicted Risk Group: Yüksek


In [23]:
def predict_anksiyete(new_point, encoder_path, model_path):

    new_data = pd.DataFrame([new_point])
    
    # Load the saved encoder
    with open(encoder_path, 'rb') as f:
        encoder = pickle.load(f)

    # Load the saved model
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    # Select only the same categorical columns used during training
    categorical_cols = new_data.select_dtypes(include=['object', 'category']).columns

    # Encode the categorical part
    encoded_new_cat = encoder.transform(new_data[categorical_cols])

    # Convert to DataFrame
    encoded_new_cat_df = pd.DataFrame(encoded_new_cat, columns=encoder.get_feature_names_out(categorical_cols), index=new_data.index)

    # Drop original categorical columns and concatenate encoded ones
    new_data_encoded = pd.concat([encoded_new_cat_df,new_data.drop(columns=categorical_cols)], axis=1)

    prediction = model.predict(new_data_encoded)

    return prediction[0]

In [24]:
encoder_path = './encoders/anksiyete_rf_encoder.pkl'
model_path = './models/anksiyete_rf_model.pkl'

In [25]:
predict_anksiyete(new_point, encoder_path, model_path)

'Yüksek'

In [12]:
predict(new_point, './encoders/anger_rf_encoder.pkl', './models/anger_rf_model.pkl')

'Yüksek Risk'